In [7]:
args = {
    'image_path': 'data/raw/image.jp2',
    'mask_path': 'data/raw/mask.jp2',
    'patch_size': 512,
    'path_to_save': 'data',
    'save_images': True,
    'mean_std_path': 'cashe/mean_std.pth',
    'test_ratio' : 0.2,
    'device': 'cuda',
    'batch_size': 4,
    'num_workers': 0,
    'lr': 1e-2,
    'momentum': 0.9,
    'weight_decay': 1e-3,
    'epochs': 50,
    'encoder_name': 'efficientnet-b0',
    'encoder_weights': 'imagenet'
}

In [8]:
from train import main

In [9]:
main(args)

Mean and std loaded from cashe.


0it [00:00, ?it/s]

Mean and std loaded from cashe.


0it [00:00, ?it/s]

  0%|          | 0/88 [00:00<?, ?it/s]

...

  0%|          | 0/88 [00:00<?, ?it/s]

  0%|          | 0/23 [00:00<?, ?it/s]

EPOCH 4| TRAIN LOSS: 1.268|  TEST LOSS: 1.248|  TEST_PRECISION: 0.000| TEST_RECALL: 0.000| TEST_ACCURACY: 0.992| TEST_F1: 0.000| TEST_IOU: 0.000| 


  0%|          | 0/88 [00:00<?, ?it/s]

KeyboardInterrupt: 